# import library

In [52]:
!pip install surprise
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# data preprocessing

In [53]:
#import dataset
ratings_data = pd.read_csv('ratings.csv')
movies_data = pd.read_csv('movies.csv')
#menggabungkan data movies dan data ratings sesuai nilai ISBN
data = pd.merge(ratings_data, movies_data, on='movieId')
df=data.copy()
#menghapus data kosong
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
#menghapus kolom yang tidak di gunakan
df.drop(columns=["timestamp"],axis=1,inplace=True)
#membuat tabel matrix item_userr
item_userr=df.pivot_table(index=["title"],columns=["userId"],values="rating")
item_userr.fillna(0,inplace=True)
# Save the DataFrame to a CSV file
data_merge = pd.DataFrame(df)
data_merge.to_csv('merge_data.csv', index=False)

# data eksploration

In [54]:
movies_data

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [55]:
ratings_data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [56]:
data_merge

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [57]:
itemuser=item_userr.head(100)
item_userr.to_csv("item-userr.csv",index=False)
item_userr

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5


# get popularity movie

In [58]:
def get_popular_movies(data, jumlah):
    """
    Mengembalikan daftar film yang paling populer berdasarkan rata-rata rating.

    Parameters:
    - data: DataFrame yang berisi data rating film.
    - n: Jumlah film yang ingin ditampilkan.

    Returns:
    DataFrame: DataFrame berisi informasi film-film yang paling populer.
    """
    popular_movies = data.groupby('movieId')['rating'].mean().sort_values(ascending=False).index
    popular_movies = popular_movies[:jumlah]

    # Dapatkan informasi tambahan tentang film dari data movies
    popular_movies_info = pd.merge(pd.DataFrame({'movieId': popular_movies}), movies_data, on='movieId')

    # Tambahkan kolom similarity dan atur nilainya menjadi 1
    popular_movies_info['similarity'] = 1

    # Tambahkan kolom pred_rating dan atur nilainya menjadi nilai rating
    popular_movies_info = pd.merge(popular_movies_info, data[['movieId', 'rating']], on='movieId')
    popular_movies_info.rename(columns={'rating': 'pred_rating'}, inplace=True)

    return popular_movies_info[['movieId', 'similarity', 'title', 'pred_rating']]


# item-based collaborative filtering

inisialisasi dataset dan model

In [ ]:
# Inisialisasi pembaca (Reader) untuk dataset Surprise
reader = Reader(rating_scale=(1, 5))

# Muat data ke dalam format Surprise
dataset = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)

# Bagi data menjadi train dan test set
trainset, testset = train_test_split(dataset, test_size=0.2)

# Inisialisasi model KNN dengan item-based collaborative filtering
sim_options = {
    'name': 'cosine',  # Pengukuran kesamaan (bisa juga 'pearson')
    'user_based': False  # Gunakan item-based collaborative filtering
}

model = KNNBasic(k=50,sim_options=sim_options)

# Latih model pada data pelatihan
model.fit(trainset)

matriks similarity

In [ ]:
# Dapatkan matriks similarity antar item dari model
item_similarity_matrix = model.sim

# Konversi matriks similarity ke array NumPy
item_similarity_matrix_array = np.array(item_similarity_matrix)

datamatriks=pd.DataFrame(item_similarity_matrix_array)

matrix=datamatriks.head(100)
# Tampilkan matriks similarity
print("Matriks Similarity Antar Item:")
print(item_similarity_matrix_array)
matrix.to_csv("matrix_cosine_.csv",index=False)

fungsi rekomendasi by input userid



In [ ]:

# Fungsi untuk merekomendasikan film untuk pengguna tertentu
def get_top_n_recommendations_by_user(user_id, n):
    # Latih model pada data pelatihan
    model.fit(trainset)

    # Dapatkan similarity matrix setelah melatih model
    similarity_matrix = model.sim
    predictions = model.test(testset)
    top_n = {}

    # Ambil daftar film yang ada dalam data pelatihan
    train_movie_ids = set(model.trainset.to_raw_iid(iid) for iid in model.trainset.all_items())

    for uid, iid, true_r, est, _ in predictions:
        # Pastikan item (film) ada dalam data pelatihan
        if iid not in train_movie_ids:
            continue

        if uid not in top_n:
            top_n[uid] = []
        top_n[uid].append((iid, est))

    # Urutkan rekomendasi berdasarkan estimasi rating tertinggi
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    # Buat DataFrame rekomendasi dengan kolom 'movieId', 'pred_rating'
    recommendations_df = pd.DataFrame(top_n[user_id], columns=['movieId', 'pred_rating'])

    # Ambil similarity hanya untuk film yang ada dalam rekomendasi
    similarities = [similarity_matrix[model.trainset.to_inner_iid(movie_id), model.trainset.to_inner_iid(movie_id)]
                    for movie_id in recommendations_df['movieId']]
    recommendations_df['similarity'] = similarities

    # Gabungkan dengan informasi film untuk mendapatkan kolom 'title'
    recommendations_user = pd.merge(recommendations_df, movies_data[['movieId', 'title']], on='movieId')
    # Jika rekomendasi dari model berbasis pengguna kurang dari n, lengkapi dengan film populer
    if len(recommendations_user) < n:
        remaining_count = n - len(recommendations_user)
        popular_movies = get_popular_movies(data, remaining_count)
        recommendations_user = pd.concat([recommendations_user, popular_movies], ignore_index=True)

    return recommendations_user[['movieId', 'similarity', 'title', 'pred_rating']]

fungsi rekomendasi by input movie title


In [ ]:
# Fungsi untuk merekomendasikan film berdasarkan judul
def get_top_n_recommendations_by_title(title, n_jumlah):
    # Latih model pada data pelatihan
    model.fit(trainset)
    try:
        movie_id = data[data['title'] == title]['movieId'].iloc[0]

        # Check if the movie is in the training set
        if movie_id not in model.trainset.all_items():
            print("error ")
            popular_movie=get_popular_movies(data, jumlah=n_jumlah)
            return popular_movie
        #Dapatkan ID film dari judul
        movie_inner_id = model.trainset.to_inner_iid(movie_id)
        # Dapatkan rekomendasi berdasarkan ID film
        raw_recs = model.get_neighbors(movie_inner_id, k=20)

        # Konversi ID film ke judul dan tambahkan pred_rating dan similarity
        movie_recs = [
            (
                model.trainset.to_raw_iid(inner_id),
                model.sim[movie_inner_id, inner_id],
                model.predict(model.trainset.to_raw_uid(movie_inner_id), model.trainset.to_raw_iid(inner_id)).est
            )
            for inner_id in raw_recs
        ]
        movie_recs_df = pd.DataFrame(movie_recs, columns=['movieId', 'similarity', 'pred_rating'])
        # Gabungkan dengan data movie untuk mendapatkan judul
        recommendations_title = pd.merge(movie_recs_df, movies_data[['movieId', 'title']], on='movieId')
        print ("recommendations_title before: " ,len(recommendations_title))
        # Lengkapi rekomendasi dengan film populer jika kurang dari n
        if len(recommendations_title) < n_jumlah:
            num_additional_recommendations = n_jumlah - len(recommendations_title)
            popular_movies = get_popular_movies(data, jumlah=num_additional_recommendations)
            recommendations_title = pd.concat([recommendations_title, popular_movies], ignore_index=True)

        return recommendations_title
    except ValueError:
        # Default recommendation: Show popular movies
        popular_movies = get_popular_movies(data, jumlah=n_jumlah)
        return popular_movies

test rekomendasi

In [48]:
user_id=1
number_of_movie=10
get_top_n_recommendations_by_user(user_id, number_of_movie)

Computing the cosine similarity matrix...
Done computing similarity matrix.


,movieId,similarity,title,pred_rating
0,2716,1.0,Ghostbusters (a.k.a. Ghost Busters) (1984),4.674543
1,1265,1.0,Groundhog Day (1993),4.624136
2,1967,1.0,Labyrinth (1986),4.600268
3,1222,1.0,Full Metal Jacket (1987),4.574381
4,3440,1.0,Teenage Mutant Ninja Turtles III (1993),4.549983
5,3479,1.0,Ladyhawke (1985),4.549789
6,101,1.0,Bottle Rocket (1996),4.549720
7,2616,1.0,Dick Tracy (1990),4.548070
8,2571,1.0,"Matrix, The (1999)",4.525092
9,1197,1.0,"Princess Bride, The (1987)",4.525071


In [49]:
movie_title='Tombstone (1993)'
number_of_movie=10
get_top_n_recommendations_by_title( movie_title, number_of_movie)

Computing the cosine similarity matrix...
Done computing similarity matrix.


,movieId,similarity,title,pred_rating
0,88448,1,Paper Birds (Pájaros de papel) (2010),5.0
1,100556,1,"Act of Killing, The (2012)",5.0
2,143031,1,Jump In! (2007),5.0
3,143511,1,Human (2015),5.0
4,143559,1,L.A. Slasher (2015),5.0
5,6201,1,Lady Jane (1986),5.0
6,102217,1,Bill Hicks: Revelations (1993),5.0
7,102084,1,Justice League: Doom (2012),5.0
8,6192,1,Open Hearts (Elsker dig for evigt) (2002),5.0
9,145994,1,Formula of Love (1984),5.0


# evaluasi model

In [50]:
#evaluasi
# List nilai k yang ingin diuji
k_values = [5, 10,20, 30, 40, 50]

# DataFrame untuk menyimpan hasil pengujian
results_df = pd.DataFrame(columns=['k', 'MAE'])

# Loop untuk menguji model dengan nilai k yang berbeda
for k in k_values:

    # Inisialisasi model KNN dengan nilai k yang diuji
    model = KNNBasic(k=k, sim_options=sim_options)

    # Latih model pada data pelatihan
    model.fit(trainset)

    # Dapatkan similarity matrix setelah melatih model
    similarity_matrix = model.sim

    # Buat prediksi pada data uji
    predictions = model.test(testset)

    # Hitung Mean Absolute Error (MAE) untuk evaluasi
    mae = accuracy.mae(predictions)

    # Tambahkan hasil pengujian ke DataFrame
    results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)

# Tampilkan DataFrame hasil pengujian
print(results_df)



Computing the cosine similarity matrix...
Done computing similarity matrix.
MAE:  0.8272
Computing the cosine similarity matrix...


<ipython-input-50-7ea9b6dd489f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)


Done computing similarity matrix.
MAE:  0.7969
Computing the cosine similarity matrix...


<ipython-input-50-7ea9b6dd489f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)


Done computing similarity matrix.
MAE:  0.7753
Computing the cosine similarity matrix...


...

<ipython-input-50-7ea9b6dd489f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)


Done computing similarity matrix.
MAE:  0.7605
Computing the cosine similarity matrix...


<ipython-input-50-7ea9b6dd489f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)


Done computing similarity matrix.
MAE:  0.7560
      k       MAE
0   5.0  0.827170
1  10.0  0.796908
2  20.0  0.775271
3  30.0  0.767355
4  40.0  0.760549
5  50.0  0.756036


<ipython-input-50-7ea9b6dd489f>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'k': k, 'MAE': mae}, ignore_index=True)


In [51]:
results_df.to_csv("eval_k.csv",index=False)